In [18]:
import pandas as pd 
import numpy as np

import xml.etree.ElementTree as ET

In [19]:
plan=[
['P1',150,150,150,150],
['P2',100,100,100,100],
['P3',100,100,50,50]]
plan=pd.DataFrame(plan,columns=['Pr',1,  2,  3,  4  ])
plan.index=plan['Pr']
plan.drop(columns=['Pr'],inplace=True)

Periode=1 #--------------------------------------------------

In [20]:
# tree = ET.parse('XMLs\\periode_01.xml')#-----------------------------------------------
# root = tree.getroot()

# dfl=[]
# for a in root.find('warehousestock').findall('article'):
#     dfl.append(a.attrib)
# bestands_df = pd.DataFrame(dfl)

In [21]:
arbeitsplaetze=15
index_l=[4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,26,49,54,29,50,55,30,51,56,31,1,2,3]
u=['K','D','H','K','D','H','K','D','H','K','D','H','KDH','KDH','K','D','H','KDH','K','D','H','K','D','H','K','D','H','K','D','H']

combis=[ 
    [ [6,18,19,20] ],
    [ [5,21,22,23] ],
    [ [5,24],[6,25,26] ],
    [ [6,27],[7,28,29] ],
    [ [0] ],
    [ [3,16,15,14], [2,12]],
    [ [2,16,15,14,17,11,10,9,8,7,6] ],
    [ [1,6,9], [2, 11,10,8,7], [3,16,15,14 ] ],
    [ [3,11,10,8,7,6,9],[2,16,15,14 ] ],
    [ [4,0,1,2,3,4,5]],
    [ [3,0,1,2,3,4,5]],
    [ [3,11,10,8,7,6,9]],
    [ [2,11,10,8,7,6,9]],
    [ [3,12]],
    [ [3,13,17]],
]
def f(x):
    plan_dict={
        'K': plan[Periode][0],
        'D': plan[Periode][1],
        'H': plan[Periode][2],
        'KDH': plan[Periode][0]+plan[Periode][1]+plan[Periode][2]
    }
    return plan_dict[x]

main_df=pd.DataFrame(np.array([u,index_l]).T,columns=['u','NR'])
#main_df['menge']=main_df['menge'].astype(float)
main_df['menge']=main_df['u'].apply(f)


df=pd.DataFrame()
for x in range(1,arbeitsplaetze+1):

    col=np.zeros(len(index_l))

    if x<len(combis)+1:
        for a in combis[x-1]:
            val=a.pop(0)
            
            for y in a:
                col[y]=val

    df[x]=col

for x in range(1,arbeitsplaetze+1):
    main_df[x]=df[x]*main_df['menge']

In [30]:
calc_l=[]
for x in range(1,arbeitsplaetze+1):
    calc_l.append(sum(main_df[x]))

pd.DataFrame([calc_l],columns=list(range(1,arbeitsplaetze+1)))

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2100.0,1750.0,1950.0,2300.0,0.0,1750.0,2800.0,2150.0,2800.0,2800.0,2100.0,2100.0,1400.0,1050.0,2100.0
